In [178]:
#Test cases multiple_test_cases()
            
def datatype_check(expected_output, target_output, error):
    success = 0
    if isinstance(target_output, dict):
        for key in expected_output.keys():
            try:
                success += datatype_check(expected_output[key], 
                                         target_output[key], error)
            except:
                print("Error: {} in variable {}. Got {} but expected type {}".format(error,
                                                                          key, type(target_output[key]), type(expected_output[key])))
        if success == len(target_output.keys()):
            return 1
        else:
            return 0
    elif isinstance(target_output, tuple) or isinstance(target_output, list):
        for i in range(len(expected_output)):
            try: 
                success += datatype_check(expected_output[i], 
                                         target_output[i], error)
                
            except:
                print("Error: {} in variable {}. Got type: {}  but expected type {}".format(error,
                                                                          i, type(target_output[i]), type(expected_output[i])))
        if success == len(target_output):
            return 1
        else:
            return 0
                
    else:
        assert isinstance(target_output, type(expected_output))
        return 1

def shape_check(expected_output, target_output, error):
    success = 0
    if isinstance(expected_output, dict):
        for key in expected_output.keys():
            try:
                success += shape_check(expected_output[key], 
                                         target_output[key], error)
            except:
                print("Error: {} for variable {}.".format(error, key))
        if success == len(expected_output.keys()):
            return 1
        else:
            return 0
    elif isinstance(expected_output, tuple) or isinstance(expected_output, list):
        for i in range(len(expected_output)):
            try: 
                success += shape_check(expected_output[i], 
                                         target_output[i], error)
            except:
                print("Error: {} for variable {}.".format(error, i))
        if success == len(expected_output):
            return 1
        else:
            return 0
                
    else:
        if hasattr(expected_output, 'shape'):
            assert target_output.shape == expected_output.shape
        return 1
    
def equation_output_check(expected_output, target_output, error):
    success = 0
    if isinstance(expected_output, dict):
        print("inside dict")
        print(expected_output.keys())
        print(target_output.keys())
        for key in expected_output.keys():
            try:
                success += equation_output_check(expected_output[key], target_output[key], error)
            except:
                print("Error: {} for variable {}.".format(error,  key))
        if success == len(target_output.keys()):
            return 1
        else:
            return 0
    elif isinstance(expected_output, tuple) or isinstance(expected_output, list):
        print("inside tuple")
        for i in range(len(expected_output)):
            try: 
                success += equation_output_check(expected_output[i], 
                                         target_output[i], error)
                print("try tuple")
                print(success)
            except:
                print("except tuple")
                print("Error: {} for variable {}.".format(error, i))
        if success == len(target_output):
            return 1
        else:
            return 0
                
    else:
        if hasattr(expected_output, 'shape'):
            print(expected_output)
            print(target_output)
            #np.allclose(target_output, expected_output)
            np.testing.assert_array_almost_equal(target_output, expected_output)
        else:
            assert target_output == expected_output
        return 1

def multiple_test(test_cases, target):
    success = 0
    for test_case in test_cases:
        try:
            target_answer = target(*test_case['input'])
            parameters, grads, v, beta, learning_rate = test_case['input']
            print("testcase")
            print(parameters)
            print(grads)
            print(v)
            print(beta)
            print(learning_rate)
            if test_case['name'] == "datatype_check":
                print("datatype")
                success += datatype_check(test_case['expected'], target_answer, test_case['error'])
                print(success)
            if test_case['name'] == "shape_check":
                print("shape")
                success += shape_check(test_case['expected'], target_answer, test_case['error'])
                print(success)
            if test_case['name'] == "equation_output_check":
                print("equation output")
                success += equation_output_check(test_case['expected'], target_answer, test_case['error'])
                print(success)
        except:
            print("Error: " + test_case['error'])
            
    if success == len(test_cases):
        print("\033[92m All tests passed.")
    else:
        print('\033[92m', success," Tests passed")
        print('\033[91m', len(test_cases) - success, " Tests failed")
        raise AssertionError("Not all tests were passed for {}. Check your equations and avoid using global variables inside the function.".format(target.__name__))

In [163]:
#actual function
import numpy as np

def update_parameters_with_momentum(params, grads, v, beta, learning_rate):
    L = len(params) // 2
    for l in range(L):
        # compute velocities
        v["dW" + str(l+1)] = beta * v["dW" + str(l+1)] + (1-beta) * grads["dW" + str(l+1)]
        v["db" + str(l+1)] = beta * v["db" + str(l+1)] + (1-beta) * grads["db" + str(l+1)]
        # update parameters
        params["W" + str(l+1)] = params["W" + str(l+1)] - learning_rate * v["dW" + str(l+1)]
        params["b" + str(l+1)] = params["b" + str(l+1)] - learning_rate * v["db" + str(l+1)]
        
    return params, v

In [164]:
#individual testing
def update_parameters_with_momentum_test_case():
    np.random.seed(1)
    W1 = np.random.randn(2, 3)
    b1 = np.random.randn(2, 1)
    W2 = np.random.randn(3, 3)
    b2 = np.random.randn(3, 1)

    dW1 = np.random.randn(2, 3)
    db1 = np.random.randn(2, 1)
    dW2 = np.random.randn(3, 3)
    db2 = np.random.randn(3, 1)
    parameters = {"W1": W1, "b1": b1, "W2": W2, "b2": b2}
    grads = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}
    v = {'dW1': np.array([[ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]), 'dW2': np.array([[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]), 'db1': np.array([[ 0.],
        [ 0.]]), 'db2': np.array([[ 0.],
        [ 0.],
        [ 0.]])}
    return parameters, grads, v

In [188]:
def update_parameters_with_momentum_test(target):
    
    expected_parameters = {
        'W1': np.array([[ 1.62544598, -0.61290114, -0.52907334],
                        [-1.07347112,  0.86450677, -2.30085497]]),
                 'b1': np.array([[ 1.74493465],
                        [-0.76027113]]),
                 'W2': np.array([[ 0.31930698, -0.24990073,  1.4627996 ],
                        [-2.05974396, -0.32173003, -0.38320915],
                        [ 1.13444069, -1.0998786 , -0.1713109 ]]),
                 'b2': np.array([[-0.87809283],
                        [ 0.04055394],
                        [ 0.58207317]])}
    expected_v = {'dW1': np.array([[-0.11006192,  0.11447237,  0.09015907],
                        [ 0.05024943,  0.09008559, -0.06837279]]),
                  'db1': np.array([[-0.01228902],
                        [-0.09357694]]),
                 'dW2': np.array([[-0.02678881,  0.05303555, -0.06916608],
                        [-0.03967535, -0.06871727, -0.08452056],
                        [-0.06712461, -0.00126646, -0.11173103]]),
                 'db2': np.array([[0.02344157],
                        [0.16598022],
                        [0.07420442]])}
    
    parameters, grads, v = update_parameters_with_momentum_test_case()
    print("second")
    print(parameters)
    print(grads)
    print(v)
    beta = 0.9
    learning_rate = 0.01

    expected_output = (expected_parameters, expected_v)
    test_cases = [
        {
            "name": "equation_output_check",
            "input": [parameters, grads, v, beta, learning_rate],
            "expected": expected_output,
            "error": "Wrong output"
        },
        {
            "name":"datatype_check",
            "input": [parameters, grads, v, beta, learning_rate],
            "expected": expected_output,
            "error":"Datatype mismatch"
        },
        {
            "name": "shape_check",
            "input": [parameters, grads, v, beta, learning_rate],
            "expected": expected_output,
            "error": "Wrong shape"
        }   
    ]
    
    multiple_test(test_cases, target)

In [189]:
parameters, grads, v = update_parameters_with_momentum_test_case()

print("first")
print(parameters)
print(grads)
print(v)



parameters, v = update_parameters_with_momentum(parameters, grads, v, beta = 0.9, learning_rate = 0.01)

update_parameters_with_momentum_test(update_parameters_with_momentum)

# print("third")
# print(parameters)

first
{'W1': array([[ 1.62434536, -0.61175641, -0.52817175],
       [-1.07296862,  0.86540763, -2.3015387 ]]), 'b1': array([[ 1.74481176],
       [-0.7612069 ]]), 'W2': array([[ 0.3190391 , -0.24937038,  1.46210794],
       [-2.06014071, -0.3224172 , -0.38405435],
       [ 1.13376944, -1.09989127, -0.17242821]]), 'b2': array([[-0.87785842],
       [ 0.04221375],
       [ 0.58281521]])}
{'dW1': array([[-1.10061918,  1.14472371,  0.90159072],
       [ 0.50249434,  0.90085595, -0.68372786]]), 'db1': array([[-0.12289023],
       [-0.93576943]]), 'dW2': array([[-0.26788808,  0.53035547, -0.69166075],
       [-0.39675353, -0.6871727 , -0.84520564],
       [-0.67124613, -0.0126646 , -1.11731035]]), 'db2': array([[0.2344157 ],
       [1.65980218],
       [0.74204416]])}
{'dW1': array([[0., 0., 0.],
       [0., 0., 0.]]), 'dW2': array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), 'db1': array([[0.],
       [0.]]), 'db2': array([[0.],
       [0.],
       [0.]])}
second
{'W1': array(

In [ ]:
W1
1.62434536, -0.61175641, -0.52817175 in 
1.62544598, -0.61290114, -0.52907334 exp

1.62544598, -0.61290114, -0.52907334 out shape, success 1

1.62544598, -0.61290114, -0.52907334 out datatype, success 1

 1.62544598, -0.61290114, -0.52907334 out equation, success 1

1.62544598, -0.61290114, -0.52907334 - out datatype , together shape and datatype
1.62753716, -0.61507611, -0.53078637 - out shape, together shape and datatype


1.62544598, -0.61290114, -0.52907334 - out datatype , together shape and datatype
1.62753716, -0.61507611, -0.53078637 - out shape, together shape and datatype

1.63051984, -0.61817831, -0.53322968 - out equation, together shape, datatype, equation



[ 1.63051984 -0.61817831 -0.53322968]